<a href="https://colab.research.google.com/github/ashishagg70/ContextBasedQA/blob/master/ContextBasedQAV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://data.deepai.org/squad1.1.zip"

--2021-05-13 05:40:01--  https://data.deepai.org/squad1.1.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9152254 (8.7M) [application/x-zip-compressed]
Saving to: ‘squad1.1.zip’

squad1.1.zip        100%[===================>]   8.73M  7.43MB/s    in 1.2s    

2021-05-13 05:40:03 (7.43 MB/s) - ‘squad1.1.zip’ saved [9152254/9152254]



In [2]:
!unzip squad1.1.zip

Archive:  squad1.1.zip
  inflating: dev-v1.1.json           
  inflating: train-v1.1.json         


In [3]:
! gdown --id 1JsjXsyXcYgAtOk6v3KijKOrAjJ_6LQpO #saved_model

Downloading...
From: https://drive.google.com/uc?id=1JsjXsyXcYgAtOk6v3KijKOrAjJ_6LQpO
To: /content/bidaf_model.tar.gz
2.42GB [00:29, 81.9MB/s]


In [4]:
!gdown --id 1-5OVwvJzf-OqxxTdDnUtArXtaFgPWzoX
!gdown --id 1FnFql2JCUHXlWsQDD-980CqOFXUxD0Ex

Downloading...
From: https://drive.google.com/uc?id=1-5OVwvJzf-OqxxTdDnUtArXtaFgPWzoX
To: /content/bidaf_model_state_dict
166MB [00:03, 47.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FnFql2JCUHXlWsQDD-980CqOFXUxD0Ex
To: /content/bidaf_model_3
166MB [00:02, 63.5MB/s]


In [6]:
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import GloVe
import json
import tqdm

import nltk
nltk.download('punkt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
cuda


In [7]:
file = open('train-v1.1.json','r')
file2 = open('dev-v1.1.json','r')
train_data = json.load(file)['data']
dev_data = json.load(file2)['data']

In [8]:
MAX_WORD_LENGTH = 40
D = 100

In [9]:
glove = GloVe('6B',100)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.39MB/s]                           
100%|█████████▉| 398487/400000 [00:14<00:00, 28204.35it/s]

In [10]:
char_to_index = defaultdict(int)
index_to_char = defaultdict(str)
# word_to_index = defaultdict(int)
# index_to_word = defaultdict(str)

num_characters = 0
num_words = len(glove)

In [ ]:
# i=1
# for key in glove.itos:
#     word_to_index[key]=i
#     index_to_word[i]=key
#     i+=1

In [11]:
def char_encode(text, is_train=True):
    global char_to_index
    global index_to_char
    global num_characters

    text_endcoding = []
    for word in text:
        encoding = np.zeros(MAX_WORD_LENGTH)
        try:
            i=0
            for char in word:
                encode = char_to_index[char]
                if is_train == True and encode == 0:
                    index = num_characters+1
                    char_to_index[char]= index
                    index_to_char[index]= char
                    encode = index
                    num_characters+=1
                encoding[i]=encode
                i+=1
        except:
            print(word)
        text_endcoding.append(encoding)
    return text_endcoding

In [12]:
def convert_to_lower(text):
    return text.lower()

def perform_word_tokenization(text):
    return word_tokenize(text)

def word_encode(text):
    global word_to_index
    global index_to_word
    global num_words
    encoding =[]
    for word in text:
        try:
            encode = glove.stoi[word]
        except:
            encode = num_words
        encoding.append(encode)
    return encoding


In [13]:
def text_preprocess(text, is_train=True):
    text = convert_to_lower(text)
    text = perform_word_tokenization(text)
    word_encoding = word_encode(text)
    char_encoding = char_encode(text, is_train)
    return char_encoding, word_encoding

def get_answer_indices(context,start,answer):
    context=convert_to_lower(context)
    tokens = perform_word_tokenization(context[:(start+1)])
    ans_tokens = perform_word_tokenization(convert_to_lower(answer))
    start = len(tokens)-1
    return start, start + len(ans_tokens)-1

In [14]:
def get_data(data,is_train = True):
    num_contexts = 0
    num_questions = 0
    char_contexts = []
    char_questions = []
    word_contexts=[]
    word_questions = []
    answers = []
    input_data = []
    for obj in data:
        for para in obj['paragraphs']:
            cce, cwe = text_preprocess(para['context'],is_train)
            char_contexts.append(cce)
            word_contexts.append(cwe)
            num_contexts+=1
            for qa in para['qas']:
                qce, qwe = text_preprocess(qa['question'],is_train)
                char_questions.append(qce)
                word_questions.append(qwe)
                num_questions+=1
                input_data.append((num_contexts-1,num_questions-1))
                ans = []
                for a in qa['answers']:
                    start,end = get_answer_indices(para['context'],a['answer_start'],a['text'])
                    ans.append([start,end])
                answers.append(ans)
    return input_data,char_contexts,char_questions,word_contexts,word_questions,answers

In [15]:
train_input_data,train_char_contexts,train_char_questions,train_word_contexts,train_word_questions,train_answers = get_data(train_data)

100%|█████████▉| 398487/400000 [00:30<00:00, 28204.35it/s]

In [16]:
dev_input_data,dev_char_contexts,dev_char_questions,dev_word_contexts,dev_word_questions,dev_answers = get_data(dev_data,False)

In [17]:
char_vocab_size = len(char_to_index)+1
word_vocab_size = num_words+1

In [18]:
class CharEmbedding(nn.Module):

    def __init__(self,vocab_size,embedding_dim = 8, cnn_kernel_size = 5,word_embedding_size =100 ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim,padding_idx=0,)
        self.cnn = nn.Conv1d(in_channels=embedding_dim,out_channels=word_embedding_size,kernel_size=cnn_kernel_size)
        self.maxpool = nn.MaxPool1d(kernel_size = MAX_WORD_LENGTH-cnn_kernel_size+1)
    
    def forward(self,x):
        # print(x)
        # print("start",x.shape)
        batch_size = x.shape[0]
        x = x.view(-1,MAX_WORD_LENGTH)
        x = self.embedding(x)
        x = x.transpose(1,2)
        x = self.cnn(x)
        x = F.relu(x)
        # print("cnn",x.shape)
        x = self.maxpool(x)
        # print("pool",x.shape)
        #TODO : ReLU
        x = x.view(batch_size,-1,x.shape[1])
        # print("final",x.shape)
        return x

In [19]:
class WordEmbedding(nn.Module):
    def __init__(self):
        super().__init__()      
        self.embedding = nn.Embedding.from_pretrained(torch.cat((glove.vectors,torch.zeros(1,glove.dim)),dim=0))

    def forward(self,x):
        x=self.embedding(x)
        return x

In [20]:
class HighwayNetworkLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.transform = nn.Sequential(
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.ReLU(),
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.ReLU()
        )
        self.gate = nn.Sequential(
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.Sigmoid()
        )
    def forward(self,x):
        x_transformed = self.transform(x)
        p = self.gate(x)
        return p*x_transformed + (1-p)*x

In [21]:
class ContextualEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(glove.dim+D,D,1,bidirectional = True,batch_first = True)

    def forward(self,x):
        x,_ = self.bilstm(x)
        return x

In [22]:
class AttentionFlowLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Linear(6*D,1)
    
    def forward(self,H,U):
        # H : contextual embedding of context
        # U : contextual embedding of query

        T = H.shape[1]
        J = U.shape[1]

        H_interleaved = torch.repeat_interleave(H,J,dim=1)
        U_repeated = U.repeat(1,T,1)

        assert(H_interleaved.shape==U_repeated.shape)

        HU = torch.cat((H_interleaved,U_repeated,H_interleaved*U_repeated),dim=-1)
        S = self.alpha(HU)
        S = S.view(-1,T,J)
        C2Q_att = F.softmax(S,dim = -1)
        U_tilde = torch.matmul(C2Q_att,U)
        Q2C_att = F.softmax(torch.max(S,dim=-1)[0],dim=-1)
        Q2C_att = Q2C_att.unsqueeze(1)
        H_tilde = torch.matmul(Q2C_att,H).repeat(1,T,1)
        G = torch.cat((H,U_tilde,H*U_tilde,H*H_tilde),dim=-1)
        return G,S


In [23]:
class ModellingLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(8*D,D,2,bidirectional = True,batch_first = True)
    def forward(self,G):
        M,_ = self.bilstm(G)
        return M

In [24]:
class OutputLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = nn.Linear(10*D,1)
        self.dense2 = nn.Linear(10*D,1)
        self.bilstm = nn.LSTM(2*D,D,1,bidirectional = True,batch_first = True)
        self.softmax = nn.LogSoftmax(dim=-1)
    def forward(self,G,M):
        GM = torch.cat((G,M),dim=-1)
        temp_GM = self.dense1(GM).squeeze(-1)
        start = self.softmax(temp_GM)
        M2,_ = self.bilstm(M)
        GM2= torch.cat((G,M2),dim=-1)
        temp_GM2 = self.dense2(GM2).squeeze(-1)
        end = self.softmax(temp_GM2)

        return start,end

In [25]:
class BiDAF(nn.Module):
    def __init__(self):
        super().__init__()
        self.char_emb_layer=CharEmbedding(char_vocab_size,embedding_dim=8)
        self.word_emb_layer=WordEmbedding()
        self.highway = HighwayNetworkLayer()
        self.cont_emb_layer=ContextualEmbedding()
        self.att_layer = AttentionFlowLayer()
        self.modelling_layer = ModellingLayer()
        self.output_layer = OutputLayer()
    
    def forward(self,context_char,context_word,query_char,query_word):
        context_char_emb = self.char_emb_layer(context_char)
        context_word_emb = self.word_emb_layer(context_word)
        final_context_word_embedding = torch.cat((context_char_emb,context_word_emb),dim = -1)
        final_context_word_embedding = self.highway(final_context_word_embedding)
        context_cont_emb = self.cont_emb_layer(final_context_word_embedding)

        query_char_emb = self.char_emb_layer(query_char)
        query_word_emb = self.word_emb_layer(query_word)
        final_query_word_embedding = torch.cat((query_char_emb,query_word_emb),dim = -1)
        # final_query_word_embedding = self.highway(final_query_word_embedding)
        query_cont_emb = self.cont_emb_layer(final_query_word_embedding)

        g,q2c_att = self.att_layer(context_cont_emb,query_cont_emb)
        m = self.modelling_layer(g)
        o = self.output_layer(g,m)
        return o,q2c_att

In [ ]:
# char_batch_context = [char_contexts[0],char_contexts[0]]
# char_batch_query = [char_questions[0],char_questions[1]]
# word_batch_context=[word_contexts[0],word_contexts[0]]
# word_batch_query = [word_questions[0],word_questions[1]]

In [ ]:
# char_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_context],batch_first=True))
# char_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_query],batch_first=True))
# word_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_context],batch_first=True))
# word_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_query],batch_first=True))

In [ ]:
def collate_fn_train(idx):
    context_char,context_word,query_char,query_word,ans_start, ans_end,num_ans = [],[],[],[],[],[],[]
    for id in idx:
        c,q = train_input_data[id]
        context_char.append(torch.LongTensor(train_char_contexts[c]))
        context_word.append(torch.LongTensor(train_word_contexts[c]))
        query_char.append(torch.LongTensor(train_char_questions[q]))
        query_word.append(torch.LongTensor(train_word_questions[q]))
        start,end = [],[]
        for answer in train_answers[q]:
            start.append(answer[0])
            end.append(answer[1])
        ans_start.append(torch.LongTensor(start))
        ans_end.append(torch.LongTensor(end))
        num_ans.append(len(start))
    
    context_char = torch.LongTensor(nn.utils.rnn.pad_sequence(context_char,batch_first=True))
    context_word = torch.LongTensor(nn.utils.rnn.pad_sequence(context_word,batch_first=True))
    query_char = torch.LongTensor(nn.utils.rnn.pad_sequence(query_char,batch_first=True))
    query_word = torch.LongTensor(nn.utils.rnn.pad_sequence(query_word,batch_first=True))
    ans_start = torch.LongTensor(nn.utils.rnn.pad_sequence(ans_start,batch_first=True))
    ans_end = torch.LongTensor(nn.utils.rnn.pad_sequence(ans_end,batch_first=True))
    num_ans = torch.LongTensor(num_ans)

    return context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx


def collate_fn_dev(idx):
    context_char,context_word,query_char,query_word,ans_start, ans_end, num_ans = [],[],[],[],[],[],[]
    for id in idx:
        c,q = dev_input_data[id]
        context_char.append(torch.LongTensor(dev_char_contexts[c]))
        context_word.append(torch.LongTensor(dev_word_contexts[c]))
        query_char.append(torch.LongTensor(dev_char_questions[q]))
        query_word.append(torch.LongTensor(dev_word_questions[q]))
        start,end = [],[]
        for answer in dev_answers[q]:
            start.append(answer[0])
            end.append(answer[1])
        ans_start.append(torch.LongTensor(start))
        ans_end.append(torch.LongTensor(end))
        num_ans.append(len(start))
    
    context_char = torch.LongTensor(nn.utils.rnn.pad_sequence(context_char,batch_first=True))
    context_word = torch.LongTensor(nn.utils.rnn.pad_sequence(context_word,batch_first=True))
    query_char = torch.LongTensor(nn.utils.rnn.pad_sequence(query_char,batch_first=True))
    query_word = torch.LongTensor(nn.utils.rnn.pad_sequence(query_word,batch_first=True))
    ans_start = torch.LongTensor(nn.utils.rnn.pad_sequence(ans_start,batch_first=True))
    ans_end = torch.LongTensor(nn.utils.rnn.pad_sequence(ans_end,batch_first=True))
    num_ans = torch.LongTensor(num_ans)

    return context_char,contexthttps://drive.google.com/drive/folders/1ERLTh5GRCUtJqyWst17yxLvUDox3gA_G?usp=sharing_word,query_char,query_word,ans_start,ans_end,num_ans,idx

In [ ]:
def EM(candidate_start,candidate_end,reference_starts,reference_ends,context,num_ans):

    for ref in zip(reference_starts[:num_ans],reference_ends[:num_ans]):

        if candidate_end-candidate_start != ref[1] - ref[0]:
            continue
        flag = 0
        for word_can,word_ref in zip(context[ref[0]:ref[1]+1],context[candidate_start:candidate_end+1]):
            if word_can != word_ref :
                flag = 1
                break
        if not flag:
            return 1
    
    return 0

def f1(candidate_start,candidate_end,reference_starts,reference_ends,context,num_ans):
    best = 0
    for ref in zip(reference_starts[:num_ans],reference_ends[:num_ans]):
        pred = Counter(context[candidate_start:candidate_end+1])
        gold = Counter(context[ref[0]:ref[1]+1])
        common = pred & gold
        if sum(common.values())== 0:
            continue
        prec = sum(common.values())/sum(pred.values())
        recall = sum(common.values())/sum(gold.values())
        best = max((2*prec*recall)/(prec+recall),best)
    return best

def EM_batch(ref_ans_starts,ref_ans_ends,num_anss,can_ans_starts,can_ans_ends,contexts):
    sum = 0
    for ref_ans_start,ref_ans_end,can_ans_start,can_ans_end,context,num_ans in zip(ref_ans_starts,ref_ans_ends,can_ans_starts,can_ans_ends,contexts,num_anss):
        sum+=EM(can_ans_start,can_ans_end,ref_ans_start,ref_ans_end,context,num_ans)
    return sum

def f1_batch(ref_ans_starts,ref_ans_ends,num_ans,can_ans_starts,can_ans_ends,contexts):
    sum = 0
    for ref_ans_start,ref_ans_end,can_ans_start,can_ans_end,context,num_ans in zip(ref_ans_starts,ref_ans_ends,can_ans_starts,can_ans_ends,contexts,num_ans):
        sum+=f1(can_ans_start,can_ans_end,ref_ans_start,ref_ans_end,context,num_ans)
    return sum/len(ref_ans_starts)


def get_best_answer(start,end):
    length = len(start)
    # arg = np.full(length,length-1)
    # max = end[-1]
    # for i in range(length-2,-1,-1):
    #     if end[i]>max:
    #         max= end[i]
    #         arg[i] = i
    #     else:
    #         arg[i]=arg[i+1]
            
    # s,e = 0,arg[0]
    # max = start[0]*end[arg[0]]
    # for i in range(1,length):
    #     p = start[i] * end[arg[i]]
    #     if max < p:
    #         max = p
    #         s = i
    #         e = arg[i]
    # return s,e 

    mat = torch.zeros(length,length)
    mat+=start.detach().cpu()
    mat=torch.tril(mat)
    mat*=end.detach().cpu().view(length,1)
    arg_m = np.argmax(mat)
    return arg_m%length,arg_m//length




def get_best_answer_batch(starts,ends):
    ss,es=[],[]
    for start,end in zip(starts,ends):
        s,e=get_best_answer(start,end)
        ss.append(s)
        es.append(e)
    ss = torch.LongTensor(ss)
    es = torch.LongTensor(es)
    return ss.to(device),es.to(device)
    
    

In [ ]:
train_loader = torch.utils.data.DataLoader(
    range(len(train_input_data)),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_train,
    # num_workers=num_workers,
    # pin_memory=pin_memory
)

dev_loader = torch.utils.data.DataLoader(
    range(len(dev_input_data)),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_dev,
    # num_workers=num_workers,
    # pin_memory=pin_memory
)

In [ ]:
bidaf2 = BiDAF().to(device)

In [ ]:
#bidaf2= torch.load('bidaf_model_3')

In [ ]:
#torch.save(bidaf2.state_dict(), 'bidaf_model_state_dict_ashish')

In [ ]:
bidaf2.load_state_dict(torch.load('bidaf_model_state_dict'))

<All keys matched successfully>

In [ ]:
batch = next(iter(train_loader))

In [ ]:
epochs = 20

learning_rate = 0.1

print_every = 50

optimizer = torch.optim.Adadelta(bidaf.parameters(),lr=learning_rate)

criterion = nn.NLLLoss()

In [ ]:
def train():
    for epoch in range(epochs):
        train_losses = []
        train_em = []
        train_f1 = []
        dev_f1 = []
        dev_em = []
        bidaf.train()
        for batch_idx, _data in enumerate(tqdm.notebook.tqdm(iter(train_loader))):
            context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx = _data
            context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
            optimizer.zero_grad()
            (start,end),_ = bidaf(context_char,context_word,query_char,query_word)
            loss1 = criterion(start,ans_start.squeeze(-1))
            loss2 = criterion(end,ans_end.squeeze(-1))
            loss = loss1+loss2
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                train_em.append(EM_batch(ans_start,ans_end,num_ans,torch.argmax(start,dim=-1),torch.argmax(end,dim=-1),context_word)/len(idx))
                train_f1.append(f1_batch(ans_start,ans_end,num_ans,torch.argmax(start,dim=-1),torch.argmax(end,dim=-1),context_word)/len(idx))
                train_losses.append(loss.item())
        bidaf.eval()
        with torch.no_grad():
            for batch_idx, _data in enumerate(tqdm.notebook.tqdm(iter(dev_loader))):
                context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx = _data
                context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
                (start,end),_ = bidaf(context_char,context_word,query_char,query_word)
                s,e = get_best_answer_batch(torch.exp(start),torch.exp(end))
                dev_em.append(EM_batch(ans_start,ans_end,num_ans,s,e,context_word)/len(idx))
                dev_f1.append(f1_batch(ans_start,ans_end,num_ans,s,e,context_word)/len(idx))

            print("epoch = {} , train_loss = {},train_em = {},dev_em={},train_f1={},dev_f1={}".format(epoch,np.mean(train_losses),np.mean(train_em),np.mean(dev_em),np.mean(train_f1),np.mean(dev_f1)))

In [ ]:
dev_f1 = []
dev_em = []
for batch_idx, _data in enumerate(tqdm.notebook.tqdm(iter(dev_loader))):
    context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx = _data
    context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
    (start,end),_ = bidaf(context_char,context_word,query_char,query_word)
    s,e = get_best_answer_batch(torch.exp(start),torch.exp(end))
    dev_em.append(EM_batch(ans_start,ans_end,num_ans,s,e,context_word)/len(idx))
    dev_f1.append(f1_batch(ans_start.detach().cpu().numpy(),ans_end.detach().cpu().numpy(),num_ans,s.detach().cpu().numpy(),e.detach().cpu().numpy(),context_word)/len(idx))

In [ ]:
np.mean(dev_em)

0.5841389728096676

In [ ]:
context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx = batch
context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
(start,end),_ = bidaf(context_char,context_word,query_char,query_word)

In [ ]:
s,e = get_best_answer_batch(torch.exp(start),torch.exp(end))

In [ ]:
f1_batch(ans_start,ans_end,num_ans,s.detach().cpu().numpy(),e.detach().cpu().numpy(),context_word)/len(idx)

torch.Size([10])


0.0

In [ ]:
train()

KeyboardInterrupt: ignored

In [ ]:
def text_preprocess(text):
    text = convert_to_lower(text)
    text = perform_word_tokenization(text)
    word_encoding = word_encode(text)
    char_encoding = char_encode(text)
    return char_encoding, word_encoding

def get_answer_from_indices(start,end,text):
    words = perform_word_tokenization(text)
    return ' '.join(words[start:end+1])


def predict_answer(context,question):
    c_char,c_word = text_preprocess(context)
    q_char,q_word = text_preprocess(question)
    print(c_char)
    print(q_char)
    bidaf2.eval()
    (start,end),att = bidaf2(torch.LongTensor([c_char]).to(device), torch.LongTensor([c_word]).to(device), torch.LongTensor([q_char]).to(device), torch.LongTensor([q_word]).to(device))
    # print(att)
    s,e = get_best_answer_batch(torch.exp(start), torch.exp(end))
    s,e = s.detach().cpu().numpy(),e.detach().cpu().numpy()
    answer = get_answer_from_indices(s[0],e[0],context)
    return answer,att

In [ ]:
len(index_to_char)

1258

In [ ]:
context='On June 4, 2014, the NFL announced that the practice of branding Super Bowl games with Roman numerals, a practice established at Super Bowl V, would be temporarily suspended, and that the game would be named using Arabic numerals as Super Bowl 50 as opposed to Super Bowl L.'
print(len(context))
question='If Roman numerals were used in the naming of the 50th Super Bowl, which one would have been used?'
print(len(question))

274
97


In [ ]:
answer,att=predict_answer(context,question)

[array([14.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.]), array([36., 22.,  3.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.]), array([47.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.]), array([23.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.]), array([41., 42., 28., 47.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       

In [ ]:
att

NameError: ignored

In [ ]:
x=[83, 1956, 34137, 35, 180, 6, 0, 7433, 3, 0, 7966, 1819, 1991, 1, 42, 48, 54, 33, 51, 180, 188]
len(x)

21

In [ ]:
att_temp = F.softmax(att,dim=1).squeeze()

In [ ]:
answer

'Super Bowl 50'

In [ ]:
att_temp[:,0]

tensor([0.0036, 0.1398, 0.0028, 0.0066, 0.0981, 0.7415, 0.0034, 0.0043],
       device='cuda:0', grad_fn=<SelectBackward>)

In [ ]:
Singapore is small country located in Southeast Asia

In [ ]:
context_char,context_word,query_char,query_word,ans_start,ans_end,num_ans,idx = batch
context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
start,end = bidaf(context_char,context_word,query_char,query_word)

In [ ]:
torch.save(bidaf.state_dict(), "bidaf_model_state_dict")

In [ ]:
tot = 0
for param in bidaf.parameters():
    if param.requires_grad:
        tot += param.numel()
print(tot)

1584191


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mkdir 'drive/MyDrive/QAmodels'

mkdir: cannot create directory ‘drive/MyDrive/QAmodels’: File exists


In [ ]:
ls 'drive/MyDrive/QAmodels'

bidaf_model_2  bidaf_model_3


In [ ]:
cp 'bidaf_model_state_dict' 'drive/MyDrive/QAmodels'

In [ ]:
!ls -lah

In [ ]:
train_data[0]

{'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
   'qas': [{'answers': [{'answer_start': 515,
       'text': 'Saint Bernadette Soubirous'}],
     'id': '5733be284776f41900661182',
     'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
    {'answers': [{'answer_start': 188, 'text': 

In [ ]:
context_char_test,context_word_test,query_char_test,query_word_test,ans_start_test, ans_end_test=[],[],[],[],[],[]
c,q = input_data[3]
context_char_test.append(torch.LongTensor(char_contexts[c]))
context_word_test.append(torch.LongTensor(word_contexts[c]))
query_char_test.append(torch.LongTensor(char_questions[q]))
query_word_test.append(torch.LongTensor(word_questions[q]))
start_test,end_test = answers[q][0]
ans_start_test.append(start_test)
ans_end_test.append(end_test)
context_char_test = torch.LongTensor(nn.utils.rnn.pad_sequence(context_char_test,batch_first=True))
context_word_test = torch.LongTensor(nn.utils.rnn.pad_sequence(context_word_test,batch_first=True))
query_char_test = torch.LongTensor(nn.utils.rnn.pad_sequence(query_char_test,batch_first=True))
query_word_test = torch.LongTensor(nn.utils.rnn.pad_sequence(query_word_test,batch_first=True))
context_char_test,context_word_test,query_char_test,query_word_test= context_char_test.to(device),context_word_test.to(device),query_char_test.to(device),query_word_test.to(device)

In [ ]:
start,end = bidaf(context_char_test,context_word_test,query_char_test,query_word_test)
torch.argmax(start),torch.argmax(end)

(tensor(76, device='cuda:0'), tensor(82, device='cuda:0'))

In [ ]:
ans_start_test, ans_end_test

([76], [82])